In [1]:
!pip install --upgrade -q accelerate peft bitsandbytes trl py7zr auto-gptq optimum transformers datasets fsspec huggingface_hub

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import torch
import os
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GPTQConfig
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer

In [4]:
data = load_dataset("knkarthick/samsum", split="train")

In [5]:
data

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [6]:
data.features

{'id': Value(dtype='string', id=None),
 'dialogue': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None)}

In [7]:
data.to_pandas()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\nJ...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\nKim: Bad mood tbh, I was ...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\nSam: i...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
14727,13863028,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...
14728,13828570,Theresa: <file_photo>\nTheresa: <file_photo>\n...,Theresa is at work. She gets free food and fre...
14729,13819050,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...
14730,13828395,Jennifer: Dear Celia! How are you doing?\nJenn...,Celia couldn't make it to the afternoon with t...


In [8]:
data.to_pandas().iloc[:,1][0]

"Amanda: I baked  cookies. Do you want some?\nJerry: Sure!\nAmanda: I'll bring you tomorrow :-)"

In [9]:
data.to_pandas().iloc[:,2][0]

'Amanda baked cookies and will bring Jerry some tomorrow.'

In [10]:
data_df = data.to_pandas()

In [11]:
data_df.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\nJ...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\nKim: Bad mood tbh, I was ...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\nSam: i...,"Sam is confused, because he overheard Rick com..."


In [12]:
data_df["text"] = data_df[["dialogue","summary"]].apply(
    lambda x: "###Human: Summarize this following dialogue: " + str(x["dialogue"] if x["dialogue"] is not None else "") + "\n###Assistant: " + str(x["summary"] if x["summary"] is not None else ""),
    axis=1
)

In [13]:
data_df['text']

,text
0,###Human: Summarize this following dialogue: A...
1,###Human: Summarize this following dialogue: O...
2,###Human: Summarize this following dialogue: T...
3,###Human: Summarize this following dialogue: E...
4,###Human: Summarize this following dialogue: S...
...,...
14727,###Human: Summarize this following dialogue: R...
14728,###Human: Summarize this following dialogue: T...
14729,###Human: Summarize this following dialogue: J...
14730,###Human: Summarize this following dialogue: J...


In [14]:
print(data_df['text'][0])

###Human: Summarize this following dialogue: Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)
###Assistant: Amanda baked cookies and will bring Jerry some tomorrow.


In [15]:
for i in range(len(data_df.iloc[:,3].iloc[-1])):
  print(data_df['text'][i])

###Human: Summarize this following dialogue: Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)
###Assistant: Amanda baked cookies and will bring Jerry some tomorrow.
###Human: Summarize this following dialogue: Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great
###Assistant: Olivia and Olivier are voting for liberals in this election. 
###Human: Summarize this following dialogue: Tim: Hi, what's up?
Kim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating
Tim: What did you plan on doing?
Kim: Oh you know, uni stuff and unfucking my room
Kim: Maybe tomorrow I'll move my ass and do everything
Kim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies
Tim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores
Tim: It really helps
Kim: thanks, maybe I'll do that
Tim: I also like using post-its in kaban st

In [16]:
data = Dataset.from_pandas(data_df)

In [17]:
data

Dataset({
    features: ['id', 'dialogue', 'summary', 'text'],
    num_rows: 14732
})

In [18]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

In [19]:
tokenizer.pad_token = tokenizer.eos_token

In [20]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,    # Enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16,  # Use fp16 for computation
    bnb_4bit_use_double_quant=True,  # Use double quantization for memory efficiency
)

In [21]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    quantization_config=quantization_config,
    device_map="auto"  # Automatically assigns layers to available GPUs
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
model.config.use_cache=False

In [23]:
model.config.pretraining_tp=1

In [24]:
model.gradient_checkpointing_enable()

In [25]:
model = prepare_model_for_kbit_training(model)

In [26]:
# ["q_proj", "v_proj", "k_proj"] → Adds key projection (more expressive)
# ["q_proj", "v_proj", "out_proj"] → Also fine-tunes attention output


###LORA

In [27]:
peft_config = LoraConfig(
        r=16, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"]
    )

In [28]:
model = get_peft_model(model, peft_config)

In [29]:
training_arguments = TrainingArguments(
        output_dir="mistral-finetuned-samsum",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        num_train_epochs=1,
        max_steps=250,
        fp16=True,
        push_to_hub=True,
        report_to="none"
  )

In [30]:
# Create the SFTTrainer
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        args=training_arguments,
        # tokenizer=tokenizer,
    )

Converting train dataset to ChatML:   0%|          | 0/14732 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/14732 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/14732 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/14732 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [31]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
100,1.902600
200,1.809700


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1110: UserWarning: Unable to fetch remote file due to the following error 403 Client Error. (Request ID: Root=1-684e8d1d-676e506b6095fb2717892880;7e5b1782-b7a2-4719-9f54-7f7618c43840)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.1 is restricted and you are not in the authorized list. Visit https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1 to ask for access. - silently ignoring the lookup for the file config.json in mistralai/Mistral-7B-Instruct-v0.1.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:236: UserWarning: Could not find a config file in mistralai/Mistral-7B-Instruct-v0.1 - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=250, training_loss=1.8435863037109375, metrics={'train_runtime': 2423.6775, 'train_samples_per_second': 0.825, 'train_steps_per_second': 0.103, 'total_flos': 3.538763435802624e+16, 'train_loss': 1.8435863037109375})

In [33]:
trainer.push_to_hub()

/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1110: UserWarning: Unable to fetch remote file due to the following error 403 Client Error. (Request ID: Root=1-684e8d26-06c84e0c39d7a7ef5bead53a;1fe01f67-4532-4c40-a828-c179c2746628)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.1 is restricted and you are not in the authorized list. Visit https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1 to ask for access. - silently ignoring the lookup for the file config.json in mistralai/Mistral-7B-Instruct-v0.1.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:236: UserWarning: Could not find a config file in mistralai/Mistral-7B-Instruct-v0.1 - will assume that the vocabulary was not modified.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/rahulsamant37/mistral-finetuned-samsum/commit/166947075a722e59ba3dbcc1ac587f524712574b', commit_message='End of training', commit_description='', oid='166947075a722e59ba3dbcc1ac587f524712574b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rahulsamant37/mistral-finetuned-samsum', endpoint='https://huggingface.co', repo_type='model', repo_id='rahulsamant37/mistral-finetuned-samsum'), pr_revision=None, pr_num=None)

In [1]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("/content/mistral-finetuned-samsum")

In [2]:
inputs = tokenizer("""
###Human: Summarize this following dialogue: Sunny: I'm at the railway station in Chennai Karthik: No problems so far? Sunny: no, everything's going smoothly Karthik: good. lets meet there soon!
###Assistant: """, return_tensors="pt").to("cuda")


In [3]:
model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/mistral-finetuned-samsum",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=25,
    pad_token_id=tokenizer.eos_token_id
)

In [5]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)


###Human: Summarize this following dialogue: Sunny: I'm at the railway station in Chennai Karthik: No problems so far? Sunny: no, everything's going smoothly Karthik: good. lets meet there soon!
###Assistant:  Sunny is at the railway station in Chennai. Everything is going smoothly. Karthik and Sunny will meet
2.8119378089904785
